There is a monthly pipeline which will run monthly once. 
Each month we have to store the data which we are getting to the table.  
The dataframe consisting of 4 columns namely employee_id, employee_name, employee_age, and month among which employee_id is unique. 
The table should contain 5 columns namely employee_id, employee_name, employee_age, month, and status.   
Status column can contain 3 kinds of values 1. Additional, 2. Existing, 3. Missing.  
  *  If we get new employee_id then, the value of the status column should be Additional  
  *  If we get the same employee_id next month then, the value of the status column should be Existing  
  *  If the employee_id is missing in the next month then the value of the status column should be Missing. 

--------------------final output dataframe------------------------  

|-------------|-------------|--------------|----------|----------|
|employee_id |employee_name | employee_age | month_no |status    |
|------------|--------------|--------------|----------|----------|
|      dbg01 |Monkey D luffy| 21           | 4        |Existing  |
|      dbg02 |Ronovara zoro | 23           | 3        |Missing   |
|      dbg03 |prince sanji  | 23           | 4        |Existing  |
|      dbg04 |nami          | 24           | 2        |Missing   |
|      dbg05 |ussopu        | 19           | 4        |Additional|
|------------|--------------|--------------|----------|----------|

###Hint: If the id matches then, the entire row should be updated

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import lit,col,expr
from delta.tables import DeltaTable

In [0]:
data_month_1 = [
  ('dgb01', 'luffy', 20, 1), 
  ('dgb02', 'zoro', 22, 1),
  ('dgb03', 'sanji', 22, 1)
        ]
schema = StructType([
  StructField('employee_id', StringType(), True),
  StructField('employee_name', StringType(), True),
  StructField('employee_age', IntegerType(), True),
  StructField('month_no', IntegerType(), True)
])
df_month_1 = spark.createDataFrame(data_month_1, schema)
df_month_1.show()

+-----------+-------------+------------+--------+
|employee_id|employee_name|employee_age|month_no|
+-----------+-------------+------------+--------+
|      dgb01|        luffy|          20|       1|
|      dgb02|         zoro|          22|       1|
|      dgb03|        sanji|          22|       1|
+-----------+-------------+------------+--------+



In [0]:
df_month_1.write.mode("overwrite").format("delta").option("path",dbname).saveAsTable(f'{dbname}.{table}')

In [0]:
# WRITE YOUR SAVE LOGIC HERE ..
def mergeAndSaveDf(df,path,database_name,target_table_name, mergeCol):
    d1 = dict(map(lambda x: (f'target.{x}', f'source.{x}'), df.columns))
    d2=dict(map(lambda x: (f'target.{x}', f'source.{x}'), df.columns))
    d1['target.status']=lit('Existing')
    d2['target.status']=lit('Additional')
    if DeltaTable.isDeltaTable(spark,f"{path}"):
        deltaTable = DeltaTable.forPath(spark, f"{path}")
        spark.sql(f"UPDATE {database_name}.{target_table_name} SET status='missing'")
        deltaTable.alias("target").merge(
          source=df.alias("source"),
          condition=f'target.{mergeCol}=source.{mergeCol}').whenMatchedUpdate(set=d1).whenNotMatchedInsert(values=d2).execute()
    else:
        spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
        df_final = df.withColumn("status",lit("Additional"))
        df_final.write.mode("overwrite").format("delta").option("path",path).saveAsTable(f"{database_name}.{target_table_name}")

In [0]:
mergeAndSaveDf(df_month_1,'dbfs:/FileStore/delta_table2','dbo2','my_view1','employee_id')

In [0]:
%sql
select * from dbo2.my_view1

employee_id,employee_name,employee_age,month_no,status
dgb01,luffy,20,1,Additional
dgb03,sanji,22,1,Additional
dgb02,zoro,22,1,Additional


In [0]:
%sql
drop database dbo

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4076083443574051>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-4076083443574051>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("ZHJvcCBkYXRhYmFzZSBkYm8=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/

In [0]:
data_month_2 = [
  ('dgb01', 'Monkey D luffy', 21, 2), 
  ('dgb04', 'nami', 24, 2)
        ]
        
df_month_2 = spark.createDataFrame(data_month_2, schema)

# call your merge method mergeAndSaveDf(df_month_2 , tgtTable)

In [0]:
mergeAndSaveDf(df_month_2,'dbfs:/FileStore/delta_table2','dbo2','my_view1','employee_id')

In [0]:
%sql
--dbutils.fs.rm('dbfs:/FileStore/delta_table2',True)
select * from dbo2.my_view1

employee_id,employee_name,employee_age,month_no,status
dgb01,Monkey D luffy,21,2,Existing
dgb04,nami,24,2,Additional
dgb03,sanji,22,1,missing
dgb02,zoro,22,1,missing


In [0]:
data_month_3 = [
  ('dgb02', 'Ronovara zoro', 23, 3),
        ]
df_month_3 = spark.createDataFrame(data_month_3, schema)
df_month_3.show()

#call your merge method mergeAndSaveDf(df_month_3 , tgtTable)

+-----------+-------------+------------+--------+
|employee_id|employee_name|employee_age|month_no|
+-----------+-------------+------------+--------+
|      dgb02|Ronovara zoro|          23|       3|
+-----------+-------------+------------+--------+



In [0]:
mergeAndSaveDf(df_month_3,'dbfs:/FileStore/delta_table2','dbo2','my_view1','employee_id')

In [0]:
%sql
select * from dbo2.my_view1

employee_id,employee_name,employee_age,month_no,status
dgb01,Monkey D luffy,21,2,missing
dgb02,Ronovara zoro,23,3,Existing
dgb03,sanji,22,1,missing
dgb04,nami,24,2,missing


In [0]:
data_month_4 = [
  ('dgb01', 'Monkey D luffy', 21, 4), 
  ('dgb03', 'prince sanji', 23, 4),
  ('dgb05', 'ussopu', 19, 4),
        ]
df_month_4 = spark.createDataFrame(data_month_4, schema)
df_month_4.show()

#call your merge method mergeAndSaveDf(df_month_4 , tgtTable)

+-----------+--------------+------------+--------+
|employee_id| employee_name|employee_age|month_no|
+-----------+--------------+------------+--------+
|      dgb01|Monkey D luffy|          21|       4|
|      dgb03|  prince sanji|          23|       4|
|      dgb05|        ussopu|          19|       4|
+-----------+--------------+------------+--------+



In [0]:
mergeAndSaveDf(df_month_4,'dbfs:/FileStore/delta_table2','dbo2','my_view1','employee_id')

In [0]:
%sql
select * from dbo2.my_view1

employee_id,employee_name,employee_age,month_no,status
dgb01,Monkey D luffy,21,4,Existing
dgb02,Ronovara zoro,23,3,missing
dgb03,prince sanji,23,4,Existing
dgb05,ussopu,19,4,Additional
dgb04,nami,24,2,missing


In [0]:
# write Assert Logic 

In [0]:
data_month_4 = [
  ('dgb01', 'Monkey D luffy', 21, 4), 
  ('dgb03', 'prince sanji', 23, 4),
  ('dgb05', 'ussopu', 19, 4),
  ('dgb02', 'Ronovara zoro', 23, 3)
        ]
df_1 = spark.createDataFrame(data_month_4, schema)

In [0]:
mergeAndSaveDf(df_month_4,'dbfs:/FileStore/delta_table2','dbo2','my_view1','employee_id')

In [0]:
#Type 2

In [0]:
data_month_1 = [
  ('dgb01', 'luffy', 20, 1,'A'), 
  ('dgb02', 'zoro', 22, 1,'A'),
  ('dgb03', 'sanji', 22, 1,'A')
        ]
schema = StructType([
  StructField('employee_id', StringType(), True),
  StructField('employee_name', StringType(), True),
  StructField('employee_age', IntegerType(), True),
  StructField('month_no', IntegerType(), True),
  StructField('Flag', StringType(), True)
])
df_month_1 = spark.createDataFrame(data_month_1, schema)
df_month_1.show()

+-----------+-------------+------------+--------+----+
|employee_id|employee_name|employee_age|month_no|Flag|
+-----------+-------------+------------+--------+----+
|      dgb01|        luffy|          20|       1|   A|
|      dgb02|         zoro|          22|       1|   A|
|      dgb03|        sanji|          22|       1|   A|
+-----------+-------------+------------+--------+----+



In [0]:
df_month_1.write.mode("overwrite").format("delta").option("path",'dbfs:/FileStore/delta_table3').saveAsTable('scd2')

In [0]:
%sql
select * from scd2

employee_id,employee_name,employee_age,month_no,Flag
dgb01,luffy,20,1,A
dgb03,sanji,22,1,A
dgb02,zoro,22,1,A


In [0]:
customersTable
  .as("customers")
  .merge(
    stagedUpdates.as("staged_updates"),
    "customers.customerId = mergeKey")
  .whenMatched("customers.current = true AND customers.address <> staged_updates.address")
  .updateExpr(Map(                                      #Set current to false and endDate to source's effective date.
    "current" -> "false",
    "endDate" -> "staged_updates.effectiveDate"))
  .whenNotMatched()
  .insertExpr(Map(
    "customerid" -> "staged_updates.customerId",
    "address" -> "staged_updates.address",
    "current" -> "true",
    "effectiveDate" -> "staged_updates.effectiveDate",  #Set current to true along with the new address and its effective date.
    "endDate" -> "null"))
  .execute()

In [0]:
deltaTable_main = DeltaTable.forPath(spark, 'dbfs:/FileStore/delta_table3')

In [0]:
data_month_2 = [
  ('dgb01', 'luffy', 20, 1), 
  ('dgb02', 'lost zoro', 22, 1),
  ('dgb04', 'Ace', 22, 1)
        ]
schema = StructType([
  StructField('employee_id', StringType(), True),
  StructField('employee_name', StringType(), True),
  StructField('employee_age', IntegerType(), True),
  StructField('month_no', IntegerType(), True)
])
df_month_2 = spark.createDataFrame(data_month_2, schema)
df_month_2.show()

+-----------+-------------+------------+--------+
|employee_id|employee_name|employee_age|month_no|
+-----------+-------------+------------+--------+
|      dgb01|        luffy|          20|       1|
|      dgb02|    lost zoro|          22|       1|
|      dgb04|          Ace|          22|       1|
+-----------+-------------+------------+--------+



In [0]:
dbutils.fs.rm('dbfs:/FileStore/delta_table2',True)

Out[65]: True

In [0]:
 deltaTable.alias("target").merge(
          source=df.alias("source"),
          condition=f'target.{mergeCol}=source.{mergeCol}').whenMatchedUpdate(set=d1).whenNotMatchedInsert(values=d2).execute()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-300362380996602>:1
----> 1 deltaTable.alias("target").merge(
      2          source=df.alias("source"),
      3          condition=f'target.{mergeCol}=source.{mergeCol}').whenMatchedUpdate(set=d1).whenNotMatchedInsert(values=d2).execute()

NameError: name 'deltaTable' is not defined

In [0]:
deltaTable_main.alias("target").merge(
    stagedUpdates.alias("source"),
    "target.employee_id = mergeKey"
).whenMatchedUpdate(
    condition="target.Flag = 'A' AND target.employee_name != source.employee_name",
    set={
        "Flag": lit("aa")  # Set current to false and endDate to source's effective date.
    }
).whenNotMatchedInsert(
    values={
        "employee_id": "source.employee_id",
        "employee_name": "source.employee_name",
        "employee_age": "source.employee_age",
        "month_no": "source.month_no",
        "Flag": lit("A")  # Set current to true along with the new address and its effective date.
    }
).execute()

In [0]:
%sql
select * from 

employee_id,employee_name,employee_age,month_no,Flag
dgb02,lost zoro,22,1,A
dgb01,luffy,20,1,A
dgb02,zoro,22,1,aa
dgb03,sanji,22,1,A
dgb04,Ace,22,1,A


In [0]:
newAddressesToInsert = df_month_2 \
  .alias("updates") \
  .join(deltaTable_main.toDF().alias("main"), "employee_id") \
  .where("main.Flag = 'A' AND updates.employee_name <> main.employee_name")

In [0]:
print(newAddressesToInsert)

DataFrame[employee_id: string, employee_name: string, employee_age: int, month_no: int, employee_name: string, employee_age: int, month_no: int, Flag: string]


In [0]:
newAddressesToInsert.display()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-300362380996614>:1
----> 1 newAddressesToInsert.display()

File /databricks/python_shell/dbruntime/monkey_patches.py:43, in apply_dataframe_display_patch.<locals>.df_display(df, *args, **kwargs)
     39 def df_display(df, *args, **kwargs):
     40     """
     41     df.display() is an alias for display(df). Run help(display) for more information.
     42     """
---> 43     display(df, *args, **kwargs)

File /databricks/python_shell/dbruntime/display.py:83, in Display.display(self, input, *args, **kwargs)
     80         if kwargs.get('trigger'):
     81             raise Exception('Triggers can only be set for streaming queries.')
---> 83         self.add_custom_display_data("table", input._jdf)
     85 elif isinstance(input, list):
     86     self.display(self.sparkSession.createDataFrame(input))

File /databricks/pyth

In [0]:
stagedUpdates = (
  newAddressesToInsert
  .selectExpr("NULL as mergeKey", "updates.*")   # Rows for 1
  .union(df_month_2.selectExpr("employee_id as mergeKey", "*"))  # Rows for 2.
)

In [0]:
stagedUpdates.display()

mergeKey,employee_id,employee_name,employee_age,month_no
dgb01,dgb01,luffy,20,1
dgb02,dgb02,lost zoro,22,1
dgb04,dgb04,Ace,22,1
